In [8]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#
#
# Ugly workarround to increase spark.kryoserializer.buffer.max
# Only way in session to Run :
#        - treeAggregate at RowMatrix.scala:122 
#        - first at PCA.scala:43
#
sc = pyspark.SparkContext()
#conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','2047mb'),('spark.driver.memory','23g'),('spark.driver.maxResultSize','21g')])
conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','512mb')])
sc.stop()
sc = pyspark.SparkContext(conf=conf,appName="Notebook-FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-CleanData")
sqlContext = SQLContext(sc)
#
#
# Arguments
#
#import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190122"
#
input_file1_playback_fraud="hdfs:///data/staged/ott_dazn/fraud-canada-tokenizedwords/dt=*/*.json"
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
input_file2_playback_not_fraud="hdfs:///data/staged/ott_dazn/logs-archive-production/parquet/dt="+process_date+"/*.parquet"
output_file2="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords/dt="+process_date
input_file3=output_file2
#
input_file4="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date+"/*.*"
#
output_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
#
#  FILTER Non-Fraud AND LABEL
from pyspark.sql import functions as F
#
#
df2= sqlContext.read.parquet(input_file2_playback_not_fraud)
df2.printSchema()
#
df3 = df2.filter(" (message LIKE '%\"Url\":\"https://isl-ca.dazn.com/misl/v2/Playback%') AND (message LIKE '%,\"Response\":{\"StatusCode\":200,\"ReasonPhrase\":\"OK\",%') AND ( ( (message LIKE '%&Format=MPEG-DASH&%' OR message LIKE '%&Format=M3U&%') ) OR (message NOT LIKE '%\"User-Agent\":\"Mozilla/5.0,(Macintosh; Intel Mac OS X 10_12_6),AppleWebKit/605.1.75,(KHTML, like Gecko),Version/11.1.2,Safari/605.1.75\"},%')   )  ")
df3.printSchema()
df4 = df3.withColumn("messagecut", expr("substring(message, locate('|Livesport.WebApi.Controllers.Playback.PlaybackV2Controller|',message)+60 , length(message)-1)"))
#
# val regexTokenizer = new RegexTokenizer().setInputCol("messagecut").setOutputCol("words").setPattern("\\w+|").setGaps(false)
#
regexTokenizer = RegexTokenizer(minTokenLength=1, gaps=False, pattern='\\w+|', inputCol="messagecut", outputCol="words", toLowercase=True)
#
tokenized = regexTokenizer.transform(df4)
tokenized.printSchema()
tokenized.coalesce(1).write.json(output_file2)
# Tokenize NON-Fraud-LABEL
# hash the message de-duplicate those records
notfraud_file=sqlContext.read.json(input_file3)
notfraud_file.printSchema()
#
notfraud_df=notfraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(0).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
notfraud_df.printSchema()
#
df_notfraud_words = notfraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
df_notfraud_words.printSchema()
#
# FILTER FRAUD AND LABEL 
# Join with Internal Curation Data in urltopredict staged folder
# hash the message de-duplicate those records
fraud_file=sqlContext.read.json(input_file1_playback_fraud)
fraud_file.printSchema()
#
fraud_df=fraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(1).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
fraud_df.printSchema()
#
df_words = fraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
df_words.printSchema()
#
result_fraud_nofraud_words = df_words.union(df_notfraud_words)
## Register Generic Functions
# -----------------------------------------------------------------------------
# Build ngrams 75 90 n=6 
# support : https://stackoverflow.com/questions/51473703/pyspark-ml-ngrams-countvectorizer-sorted-based-on-count-weights
# -----------------------------------------------------------------------------
def build_ngrams_part(inputCol="words", n=6):
    ngrams = [ 
        NGram(n=i, inputCol="words", outputCol="ngrams_{0}".format(i)) 
        for i in range(7, n + 1) ]
    vectorizers = [ 
        CountVectorizer(inputCol="ngrams_{0}".format(i), outputCol="ngramscounts_{0}".format(i)) 
        for i in range(7, n + 1) ]
    return Pipeline(stages=ngrams + vectorizers)
#    assembler = [VectorAssembler( inputCols=["ngramscounts_{0}".format(i) for i in range(1, n + 1)], outputCol="features" )]
#    return Pipeline(stages=ngrams + DenseVector(SparseVector(vectorizers).toArray()))
#
# 
# -----------------------------------------------------------------------------
#ngram = build_ngrams_part().fit(df_words)
#ngramDataFrame = ngram.transform(df_words)
#ngramDataFrame.coalesce(1).write.json(output_file1)
#
ngram = NGram(n=7, inputCol="words", outputCol="ngrams_7")
countvector = CountVectorizer(inputCol="ngrams_7", outputCol="ngramscounts_7")
# fit a CountVectorizerModel from the corpus.
countvModel = CountVectorizer(inputCol="words", outputCol="features_85", vocabSize=85, minDF=2.0)
# fit a PCA Dimensionality reduction into 85/7=12.x components from ngramscounts_7 ## Too Heavy 1st PCA
pcaNgrams = PCA(k=5, inputCol="ngramscounts_7", outputCol="pcaweightngrams")
# fit a PCA Dimensionality reduction into 85/7=12.x components from words
pcaWords = PCA(k=5, inputCol="features_85", outputCol="pcaweightwords")  ## Too Heavy 2nd PCA
#
ngram_fraud_DF = ngram.transform(result_fraud_nofraud_words)
ngram_vc_fraud_DF = countvector.fit(ngram_fraud_DF).transform(ngram_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
ngram_vc_fraud_DF.printSchema()
#
#modelPCA_ngram_fraud_DF = pcaNgrams.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_ngram_fraud_DF.printSchema()
#
#result_ngrams_words_fraud_DF = countvModel.fit(modelPCA_ngram_fraud_DF).transform(modelPCA_ngram_fraud_DF)\
result_ngrams_words_fraud_DF = countvModel.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
result_ngrams_words_fraud_DF.printSchema()
#
result_ngrams_words_fraud_DF.coalesce(1).write.json(output_file1)
#
#  CALCULATE KL,KS COEF. Label Data
ngram7_fraud=sqlContext.read.json(input_file4).filter("fraud_label=0" )
ngram7_fraud.printSchema()
#
print("Calculation of standard_fraud_ngram - Start!")
#
# Obtain the most frequent word on each position 
# Compose the standard_fraud_ngram from the most common positions
# Calculate the standard_fraud_ngram
#
# https://stackoverflow.com/questions/35218882/find-maximum-row-per-group-in-spark-dataframe 
# Using struct ordering:
#from pyspark.sql.functions import struct
#
#(cnts
#  .groupBy("id_sa")
#  .agg(F.max(struct(col("cnt"), col("id_sb"))).alias("max"))
#  .select(col("id_sa"), col("max.id_sb")))
#
#Py4JJavaError: An error occurred while calling o3400.select.
#: org.apache.spark.sql.AnalysisException: cannot resolve '`ngrams`' given input columns: [mostfrequent];;
#'Project ['ngrams, mostfrequent#8033.count AS count#8037L]
#+- AnalysisBarrier
#      +- Aggregate [max(named_struct(count, count#8029L)) AS mostfrequent#8033]
#         +- Aggregate [ngrams#8016], [ngrams#8016, count(1) AS count#8029L]
#            +- Relation[fraud_label#8013L,hash_message#8014,message#8075,ngrams#8016,words#8017] json
#
####### OLD  ######
#.groupby(col('hash_message'))\
#.agg(F.first(col('features_75')).alias('features_75'),\
#     F.first(col('fraud_label')).alias('fraud_label'),\
#     F.first(col('ngrams_75')).alias('ngrams_75'),\
#     F.first(col('ngramscounts_75')).alias('ngramscounts_75'),F.first(col('words')).alias('words'))
####################
#
most_frequent_df=ngram7_fraud\
.withColumn("value_sum",F.explode("ngramscounts_7.values"))\
.groupBy("hash_message").agg(F.sum("value_sum").alias('count'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
most_frequent_df.printSchema()
#
most_frequent_df.coalesce(1).write.json(output_most_frequent_df)
#
# The most Frequent would the the max
standard_fraud_ngram=most_frequent_df.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()
#
print("Value Print: standard_fraud_ngram=")
print(standard_fraud_ngram)
#
print("Calculation of standard_fraud_ngram - Done!")
#
sc.stop()
#
print("Preparation of Data Done!")
#

root
 |-- metadata: struct (nullable = true)
 |    |-- beat: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- logzio_id: string (nullable = true)
 |-- awsRegion: string (nullable = true)
 |-- beat: struct (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- correlationId: string (nullable = true)
 |-- function_version: string (nullable = true)
 |-- host: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- input_type: string (nullable = true)
 |-- invoked_function_arn: string (nullable = true)
 |-- it: string (nullable = true)
 |-- level: long (nullable = true)
 |-- logGroup: string (nullable = true)
 |-- logStream: string (nullable = true)
 |-- logzio_codec: string (nullable = true)
 |-- message: string (nullable = true)
 |-- messageType: string (nullable = true)
 |-- msg: s

Value Print: standard_fraud_ngram=
                                        hash_message
0  8ddb1c2d52ccffc9d4e980fa9ba306758c87e9fa8f9626...
Calculation of standard_fraud_ngram - Done!
Preparation of Data Done!


In [7]:
sc.stop()

In [14]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#  FILTER with PySpark SQL Functions F.
from pyspark.sql import functions as F
# FUNCTIONS
# 
##
## Register Generic UDF Functions
# -----------------------------------------------------------------------------
# KL : Kullback-Leibler Divergence
# KS : Kolmogorov-Smirnov ( Sample sizes can be different)
# -----------------------------------------------------------------------------    
## NGRAM hash_mesage KL from URL TO Default NGRAM KL
def func_kl_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list on same size
    def KL(P,Q):
        epsilon = 0.00001
        P = P+epsilon
        Q = Q+epsilon
        divergence = np.sum(P*np.log(P/Q))
        return np.asscalar(divergence)
    ##Making sure the analysis ignore trash
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with common Malware/phishing sites used in internal webtraffic
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on the same webdomain name
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    return KL(values1,values2)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## NGRAM hash_message "Kolmogorov-Smirnov test KS pvalue= ks_2samp function8.
def func_ks_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list on same size
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    values1= np.sort(values1)
    # URLS with up to least 4166 characters
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on the same messgae
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    values2=np.sort(values2)
    (Darray,pvalue)=stats.ks_2samp(values1, values2)
    return np.asscalar(pvalue)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## Entropy TO hash_message function3.
def func_entropy_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list ignore size
    #
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with an NGRAM search
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on point
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    pvalue=stats.entropy(values1, values2)
    scalar_pvalue=np.asscalar(pvalue)
    return scalar_pvalue
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
#
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
func_kl_ngram_msg_udf = udf(func_kl_ngram_msg, FloatType())
func_ks_ngram_msg_udf = udf(func_ks_ngram_msg, FloatType())
func_entropy_ngram_msg_udf = udf(func_entropy_ngram_msg, FloatType())
#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190122"
#
#
sc = pyspark.SparkContext()
#conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','2047mb'),('spark.driver.memory','23g'),('spark.driver.maxResultSize','21g')])
conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','512mb')])
sc.stop()
sc.stop()
sc = pyspark.SparkContext(conf=conf,appName="Notebook-FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-Labeling")
sqlContext = SQLContext(sc)
#
#
input_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
input_fraud="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
input_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/label-fraud-notfraud-data-model/dt="+process_date
preserve_training_output_file="hdfs:///data/staged/ott_dazn/advanced-model-data/preserve-training-output-automl-clean/dt="+process_date
#
# The most Frequent would the ones with the max frequency of NGrams85 tokens
pd.options.display.max_colwidth = 512
#
standard_fraud_ngram=sqlContext.read.json(input_most_frequent_df)\
.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()['hash_message'][0]
print("Value UDF : standard_fraud_ngram=")
print(standard_fraud_ngram)
#
# Select Tokens/words form the max frequency of NGrams85 tokens hash_message
standard_fraud_ngram_words=sqlContext.read.json(input_fraud)
standard_fraud_ngram_words.printSchema()
#
standard_words_search=standard_fraud_ngram_words\
.withColumn('fraud_master_hash',lit(standard_fraud_ngram).cast('string'))\
.filter(" hash_message=fraud_master_hash ")
standard_words_search.printSchema()
#
standard_words=standard_words_search\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
.select(col('words_conc')).limit(1).toPandas()['words_conc'][0] 
# Working now! 
# Needs to be bigger than message twice should be tested!
#   Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
#      : java.lang.RuntimeException: Unsupported literal type class java.util.HashMap 
# {0=root15c466c8e6e8f9d17adb73426cd55c70f72b9f18e39e3455c9043a18b86b122b6requestmethodgeturlhttpsislcadazncommislv2playbackassetidfg5oon8sl71n1nfwuegbo8npgeventidarticleidfg5oon8sl71n1nfwuegbo8npgformatmpegdashplayeriddaznf3874e050812a853securetruelanguagecodeenlatitudenulllongitudenullplatformandroidtvmanufacturernvidiamodelnullmtalanguagecodeenclientip50100225179headersuseragentmozilla50linuxandroid800shieldandroidtvbuildopr6170623010wvapplewebkit53736khtmllikegeckoversion40chrome710357899mobilesafari53736fev1420typeinresponsestatuscode200reasonphraseokduration47jwtvieweridc2ebc25d8085deviceid993bf365c72c4b0b9168c2ebc25d8085f3874e050812a853userstatusactivepaid}
#
print("Value UDF : standard_words=")
print(standard_fraud_ngram)
#
ngram7_fraud=sqlContext.read.json(input_file1)
ngram7_fraud.printSchema()
#
drop_phish_cols=['words','ngrams_7']
#
fraud_label_read_df=ngram7_fraud.filter("hash_message is not NULL")\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
    .drop(*drop_phish_cols)\
    .withColumn('kl_words',func_kl_ngram_msg_udf(col('words_conc'),lit(standard_words).cast('string')).cast('double'))\
    .withColumn('ks_words',func_ks_ngram_msg_udf(col('words_conc'),lit(standard_words).cast('string')).cast('double'))\
    .withColumn('entropy_words',func_entropy_ngram_msg_udf(col('words_conc'),lit(standard_words).cast('string')).cast('double'))\
    .persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
fraud_label_read_df.printSchema()
#
fraud_label_read_df.coalesce(1).write.json(output_file1)
#
fraud_label_read_df.unpersist()
#
sc.stop()
#
print("Model Data KL,KS, Entropy Done! NGrams Vectors Data Done!")
#

Value UDF : standard_fraud_ngram=
0d3f40b98f94ffdc7b9f2037b9ec57c791a2213029d1fc972cc615226de2169cfc268480d4d8cb92d61590f7bb86f06180c7120121ab20312200e80328c3ff08
root
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- ngrams_7: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngramscounts_7: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words: array (nullable = true)
 

In [13]:
sc.stop()

In [18]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#  FILTER with PySpark SQL Functions F.
#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190122"
#
#
sc = pyspark.SparkContext()
#conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','2047mb'),('spark.driver.memory','23g'),('spark.driver.maxResultSize','21g')])
conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','512mb'),('spark.driver.memory','16g'),('spark.driver.maxResultSize','12g')])
sc.stop()
sc.stop()
sc = pyspark.SparkContext(conf=conf,appName="Notebook-FraudCanada-AUTOML-Model-NGrams-CountVectorizer-KL-KS-Entropy")
sqlContext = SQLContext(sc)
#
input_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
input_fraud="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-5-features-85/dt="+process_date
#
input_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-5-features-85/dt="+process_date
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/label-fraud-notfraud-data-model/dt="+process_date
preserve_training_input_file="hdfs:///data/staged/ott_dazn/advanced-model-data/preserve-training-output-automl-clean/dt="+process_date
#
import h2o
from h2o.automl import H2OAutoML
#
import subprocess
subprocess.run('unset http_proxy', shell=True)
#
# Start an H2O virtual cluster that uses 6 gigs of RAM and 6 cores
h2o.init(ip="localhost",port=54321,max_mem_size = "6g", nthreads = 6) 
#
# Clean up the h2o cluster just in case
h2o.remove_all()
#
#  TRAINING PROCESS
#
print("Start Training Model NGrams Vectors KS KL Entropty")
#
# Horrible code :: close your eyes, is ugly
#
fraud_label_read_file=sqlContext.read.json(output_file1)
fraud_label_read_file.printSchema()
#
fraud_label_read_df=fraud_label_read_file\
.select(col('hash_message').cast('string'),col('fraud_label').cast('int'),\
        col('kl_words').cast('double'), col('ks_words').cast('double'),col('entropy_words').cast('double'),\
        col('features_85.type').alias('features85_type').cast('long'),\
        col('features_85.size').alias('features85_size').cast('long'),\
        col('features_85.indices').alias('features85_indices'),\
        col('features_85.values').alias('features85_values'),\
        col('ngramscounts_7.type').alias('ngramscounts7_type').cast('long'),\
        col('ngramscounts_7.size').alias('ngramscounts7_size').cast('long'),\
        col('ngramscounts_7.indices').alias('ngramscounts7_indices'),\
        col('ngramscounts_7.values').alias('ngramscounts7_values'))
fraud_label_read_df.printSchema()
#
# ABOVE ARE CASE ISSUES on struct Struct of features_85 and ngramscounts_7 
# Both cares conversion to DF valide type list
# Flat vars for each, individually and seperately from the original struct
#
# https://stackoverflow.com/questions/47401418/pyspark-conversion-to-array-types?rq=1 
#
#
fraud_fraud_label_read1_df=fraud_label_read_df.filter("fraud_label=1")\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
notfraud_fraud_label_read1_df=fraud_label_read_df.filter("fraud_label=0")\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
fraud_fraud_label_read1_df.printSchema()
notfraud_fraud_label_read1_df.printSchema()
#
drop_list_cols=['features85_indices','features85_values','ngramscounts7_indices','ngramscounts7_values']
#
### 1.) https://stackoverflow.com/questions/38610559/convert-spark-dataframe-column-to-python-list
###    list(spark_df.select('mvv').toPandas()['mvv'])
### 2.) http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html
###    df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)
### 3.) https://stackoverflow.com/questions/43216411/pandas-flatten-a-list-of-list-within-a-column
###    df['var2'] = df['var2'].apply(np.ravel)
fraud_label_train_pd=fraud_fraud_label_read1_df.limit(2500).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#.tail(1200)
# dummy_dif=lambda x:list(x['fraud_label'])
#
fraud_label_test_pd=fraud_fraud_label_read1_df.limit(500).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#
not_fraud_label_train_pd=notfraud_fraud_label_read1_df.limit(2500).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#.tail(3000)
#
not_fraud_label_test_pd=notfraud_fraud_label_read1_df.limit(500).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#.head(800)
#
not_fraud_label_train=h2o.H2OFrame(not_fraud_label_train_pd)
not_fraud_label_test=h2o.H2OFrame(not_fraud_label_test_pd)
#
fraud_label_train=h2o.H2OFrame(fraud_label_train_pd)
fraud_label_test=h2o.H2OFrame(fraud_label_test_pd)
#
################# Use Two DataFrames ##################### - rbind() H2o Frames issue
#
#
###### TRAINING PROCESS ############
# RBIND "Merge" all of vars internal subset of data with fraud and with not_fraud
# function merge() doesn't work if both H2O/dataframes have same variables
#
train = fraud_label_train.rbind(not_fraud_label_train)
test = fraud_label_test.rbind(not_fraud_label_test)
#
print("train")
print(train.head(10))
print("test")
print(test.head(10))
#
# Identify predictors and response
x = train.columns
#
# Fraud Label to be learned in the model from the atrributes of the ngram85 learned words
#
y= 'fraud_label'
x.remove(y)
#
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
#
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# Balance Classes to compensate unbalanced data
# Run AutoML for 25 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=25, seed=19, exclude_algos=["DRF","GLM"])
aml.train(x=x, y=y, training_frame=train)
#
preserve_training_output.write.json(preserve_training_output_file)
#
print("AutoML Modeling Done!")
#
sc.stop()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,7 days 23 hours 18 mins
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,3 months and 4 days
H2O cluster name:,H2O_from_python_siemanalyst_f03p4d
H2O cluster total nodes:,1
H2O cluster free memory:,4.732 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


Start Training Model NGrams Vectors KS KL Entropty
root
 |-- entropy_words: double (nullable = true)
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- kl_words: double (nullable = true)
 |-- ks_words: double (nullable = true)
 |-- ngramscounts_7: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words_conc: string (nullable = true)

root
 |-- hash_message: string (nullable = true)
 |-- fraud_label: i

hash_message,fraud_label,kl_words,ks_words,entropy_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
589ff83a30fa5d82563f0051bbd9f638d3dc7f102f8c78ab137c17b859e740de35348417e0deb2037cd0188c243c674100eb57f97450c4b942c9d3a1ef86a07b,1,5121.78,0.386449,0.0775856,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 251 252 253 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 532 1571 1572 1573 2191 11120 32829 35576 37068 51952 119101 129064 152122 153250 164794 175213 183033 186242 201741 224421 238588 259672],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
865fc00f8a94dd1c467c39753bfb2b18626f2827db85a05654caf048e657b6856afad3fef87873197e7f92e0ff760c8f04cd5484c910a037476643addfdb5c31,1,3572.81,0.999849,0.067809,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 118 119 120 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 1861 2230 2233 2234 2235 2243 2244 3891 7550 7571 7600 7674 14441 67052 83797 152373 155708 158659 159474 175973 183005 184980 186211 187030 188782 196015 199169 199626 200056],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
a659ae2c8efbde6eb990b90acae3e02f644e44cd1a3a9aa6490632209641eb5714bb4b740ddcbe95e50b5cd319a8725634ffea38ae633fd6ab1f7114ad8c17bd,1,4872.41,0.629505,0.0770424,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 2210 2212 2213 2766 3349 12663 12732 13137 35371 40445 63248 160374 163934 167319 170979 176627 185968 187302 207482 215154 242580 254804],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
d6d7bcd45dfa2fb6487352a532359f3f64afe250dacb422898ed9854af8992866bff17dac83f36763b46f2db2e0442fdf23fafdd58628636ca3395b6ef019a16,1,4971.4,0.477263,0.0765636,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14


test


hash_message,fraud_label,kl_words,ks_words,entropy_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
589ff83a30fa5d82563f0051bbd9f638d3dc7f102f8c78ab137c17b859e740de35348417e0deb2037cd0188c243c674100eb57f97450c4b942c9d3a1ef86a07b,1,5121.78,0.386449,0.0775856,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 251 252 253 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 532 1571 1572 1573 2191 11120 32829 35576 37068 51952 119101 129064 152122 153250 164794 175213 183033 186242 201741 224421 238588 259672],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
865fc00f8a94dd1c467c39753bfb2b18626f2827db85a05654caf048e657b6856afad3fef87873197e7f92e0ff760c8f04cd5484c910a037476643addfdb5c31,1,3572.81,0.999849,0.067809,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 118 119 120 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 1861 2230 2233 2234 2235 2243 2244 3891 7550 7571 7600 7674 14441 67052 83797 152373 155708 158659 159474 175973 183005 184980 186211 187030 188782 196015 199169 199626 200056],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
a659ae2c8efbde6eb990b90acae3e02f644e44cd1a3a9aa6490632209641eb5714bb4b740ddcbe95e50b5cd319a8725634ffea38ae633fd6ab1f7114ad8c17bd,1,4872.41,0.629505,0.0770424,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 2210 2212 2213 2766 3349 12663 12732 13137 35371 40445 63248 160374 163934 167319 170979 176627 185968 187302 207482 215154 242580 254804],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
d6d7bcd45dfa2fb6487352a532359f3f64afe250dacb422898ed9854af8992866bff17dac83f36763b46f2db2e0442fdf23fafdd58628636ca3395b6ef019a16,1,4971.4,0.477263,0.0765636,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14


AutoML progress: |████████████████████████████████████████████████████████| 100%


NameError: name 'preserve_training_output' is not defined

In [17]:
sc.stop()

In [19]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_20190131_132458_model_3,1,0.00125637,0,0.0175969,0.00030965
GBM_5_AutoML_20190131_132458,1,0.00296727,0,0.0226319,0.000512201
GBM_2_AutoML_20190131_132458,1,0.0028804,0,0.0209347,0.00043826
GBM_1_AutoML_20190131_132458,1,0.00301976,0,0.0223972,0.000501634
GBM_grid_1_AutoML_20190131_132458_model_2,1,0.00292153,0,0.0230052,0.00052924
XGBoost_grid_1_AutoML_20190131_132458_model_5,1,0.0625543,0,0.137422,0.0188849
GBM_grid_1_AutoML_20190131_132458_model_4,1,0.00304807,0,0.024028,0.000577346
StackedEnsemble_BestOfFamily_AutoML_20190131_132458,1,0.00202966,0,0.0166555,0.000277405
GBM_4_AutoML_20190131_132458,1,0.00316898,0.000220848,0.022934,0.000525969
GBM_3_AutoML_20190131_132458,1,0.00267538,0.000220848,0.0186747,0.000348745


In [20]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20190131_132458_model_3


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 4.672689252681684e-18
RMSE: 2.1616404078110873e-09
LogLoss: 6.55074370626418e-10
Mean Per-Class Error: 0.0
AUC: 1.0
pr_auc: 0.0007558578987150177
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999999659099943: 


,0,1,Error,Rate
0,2264.0,0.0,0.0,(0.0/2264.0)
1,0.0,1323.0,0.0,(0.0/1323.0)
Total,2264.0,1323.0,0.0,(0.0/3587.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,1.0000000,1.0,1.0
max f2,1.0000000,1.0,1.0
max f0point5,1.0000000,1.0,1.0
max accuracy,1.0000000,1.0,1.0
max precision,1.0000000,1.0,0.0
max recall,1.0000000,1.0,1.0
max specificity,1.0000000,1.0,0.0
max absolute_mcc,1.0000000,1.0,1.0
max min_per_class_accuracy,1.0000000,1.0,1.0
max mean_per_class_accuracy,1.0000000,1.0,1.0


Gains/Lift Table: Avg response rate: 36.88 %, avg score: 36.88 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,1.0,0.0,1.0,1.0,1.0,1.0,1.0
auc,1.0,0.0,1.0,1.0,1.0,1.0,1.0
err,0.0,0.0,0.0,0.0,0.0,0.0,0.0
err_count,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f0point5,1.0,0.0,1.0,1.0,1.0,1.0,1.0
f1,1.0,0.0,1.0,1.0,1.0,1.0,1.0
f2,1.0,0.0,1.0,1.0,1.0,1.0,1.0
lift_top_group,2.7112694,0.0028505,2.709434,2.709434,2.7056603,2.715909,2.715909
logloss,0.0012569,0.0013725,0.0006465,0.0000000,0.0000000,0.0005358,0.0051021
max_per_class_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2019-01-31 13:30:55,3 min 36.342 sec,0.0,0.4824883,0.6583311,0.5,0.0,1.0,0.6311681,0.4852304,0.6637654,0.5,0.0,1.0,0.6210526
,2019-01-31 13:30:55,3 min 36.381 sec,5.0,0.3338924,0.3975409,1.0,0.9969766,2.7112623,0.0,0.3374659,0.4025303,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.446 sec,10.0,0.2222338,0.2440872,1.0,0.9984883,2.7112623,0.0,0.2250764,0.2472630,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.505 sec,15.0,0.1511170,0.1583229,1.0,0.9984883,2.7112623,0.0,0.1538599,0.1610000,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.561 sec,20.0,0.1064182,0.1077223,1.0,0.9984883,2.7112623,0.0,0.1094944,0.1104652,1.0,0.9930556,2.6388889,0.0
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2019-01-31 13:30:59,3 min 40.526 sec,335.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0277778,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.593 sec,340.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0277778,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.651 sec,345.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0208333,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.721 sec,350.0,0.0000000,0.0000000,1.0,0.0007559,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0138889,2.6388889,0.0



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
features85_list_indices,2386.1770020,1.0,0.4569574
features85_list_values,1668.8812256,0.6993954,0.3195939
kl_words,580.0820923,0.2431010,0.1110868
ks_words,257.0541687,0.1077264,0.0492264
ngramscounts7_list_values,221.1662292,0.0926864,0.0423537
entropy_words,108.5201645,0.0454787,0.0207818


In [21]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.head(10))
print("prediction")
print(preds.head(10))

gbm prediction progress: |████████████████████████████████████████████████| 100%
test


hash_message,fraud_label,kl_words,ks_words,entropy_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
589ff83a30fa5d82563f0051bbd9f638d3dc7f102f8c78ab137c17b859e740de35348417e0deb2037cd0188c243c674100eb57f97450c4b942c9d3a1ef86a07b,1,5121.78,0.386449,0.0775856,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 251 252 253 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 532 1571 1572 1573 2191 11120 32829 35576 37068 51952 119101 129064 152122 153250 164794 175213 183033 186242 201741 224421 238588 259672],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
865fc00f8a94dd1c467c39753bfb2b18626f2827db85a05654caf048e657b6856afad3fef87873197e7f92e0ff760c8f04cd5484c910a037476643addfdb5c31,1,3572.81,0.999849,0.067809,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 118 119 120 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 1861 2230 2233 2234 2235 2243 2244 3891 7550 7571 7600 7674 14441 67052 83797 152373 155708 158659 159474 175973 183005 184980 186211 187030 188782 196015 199169 199626 200056],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
a659ae2c8efbde6eb990b90acae3e02f644e44cd1a3a9aa6490632209641eb5714bb4b740ddcbe95e50b5cd319a8725634ffea38ae633fd6ab1f7114ad8c17bd,1,4872.41,0.629505,0.0770424,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 16 66 67 68 69 70 177 178 179 180 181 182 204 271 272 312 313 365 366 367 368 369 370 372 373 374 375 376 421 422 423 424 425 426 427 428 2210 2212 2213 2766 3349 12663 12732 13137 35371 40445 63248 160374 163934 167319 170979 176627 185968 187302 207482 215154 242580 254804],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
d6d7bcd45dfa2fb6487352a532359f3f64afe250dacb422898ed9854af8992866bff17dac83f36763b46f2db2e0442fdf23fafdd58628636ca3395b6ef019a16,1,4971.4,0.477263,0.0765636,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 38 39 40 41 42 43 44 45 46 47 48 49 50 52 53 54 55 56 61 62 64 65],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14


prediction


predict,p0,p1
1,1.87768e-10,1
1,1.4861e-09,1
1,3.62737e-11,1
1,1.359e-11,1
1,3.21747e-11,1
1,3.42677e-10,1
1,1.86118e-11,1
1,2.32721e-10,1
1,9.11604e-12,1
1,1.69261e-10,1


In [23]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

print(model_ids)
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "GBM" in mid][0])
print(se)

# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(aml.leader.metalearner()['name'])
metalearner.coef_norm()

['GBM_grid_1_AutoML_20190131_132458_model_3', 'GBM_5_AutoML_20190131_132458', 'GBM_2_AutoML_20190131_132458', 'GBM_1_AutoML_20190131_132458', 'GBM_grid_1_AutoML_20190131_132458_model_2', 'XGBoost_grid_1_AutoML_20190131_132458_model_5', 'GBM_grid_1_AutoML_20190131_132458_model_4', 'StackedEnsemble_BestOfFamily_AutoML_20190131_132458', 'GBM_4_AutoML_20190131_132458', 'GBM_3_AutoML_20190131_132458', 'XGBoost_grid_1_AutoML_20190131_132458_model_3', 'XGBoost_grid_1_AutoML_20190131_132458_model_1', 'DeepLearning_grid_1_AutoML_20190131_132458_model_3', 'StackedEnsemble_AllModels_AutoML_20190131_132458', 'DeepLearning_1_AutoML_20190131_132458', 'XGBoost_grid_1_AutoML_20190131_132458_model_2', 'DeepLearning_grid_1_AutoML_20190131_132458_model_4', 'XGBoost_1_AutoML_20190131_132458', 'DeepLearning_grid_1_AutoML_20190131_132458_model_2', 'XGBoost_grid_1_AutoML_20190131_132458_model_6', 'GBM_grid_1_AutoML_20190131_132458_model_1', 'XGBoost_2_AutoML_20190131_132458', 'DeepLearning_grid_1_AutoML_2019

,0,1,Error,Rate
0,2264.0,0.0,0.0,(0.0/2264.0)
1,0.0,1323.0,0.0,(0.0/1323.0)
Total,2264.0,1323.0,0.0,(0.0/3587.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,1.0000000,1.0,1.0
max f2,1.0000000,1.0,1.0
max f0point5,1.0000000,1.0,1.0
max accuracy,1.0000000,1.0,1.0
max precision,1.0000000,1.0,0.0
max recall,1.0000000,1.0,1.0
max specificity,1.0000000,1.0,0.0
max absolute_mcc,1.0000000,1.0,1.0
max min_per_class_accuracy,1.0000000,1.0,1.0
max mean_per_class_accuracy,1.0000000,1.0,1.0


Gains/Lift Table: Avg response rate: 36.88 %, avg score: 36.88 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2019-01-31 13:30:55,3 min 36.342 sec,0.0,0.4824883,0.6583311,0.5,0.0,1.0,0.6311681,0.4852304,0.6637654,0.5,0.0,1.0,0.6210526
,2019-01-31 13:30:55,3 min 36.381 sec,5.0,0.3338924,0.3975409,1.0,0.9969766,2.7112623,0.0,0.3374659,0.4025303,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.446 sec,10.0,0.2222338,0.2440872,1.0,0.9984883,2.7112623,0.0,0.2250764,0.2472630,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.505 sec,15.0,0.1511170,0.1583229,1.0,0.9984883,2.7112623,0.0,0.1538599,0.1610000,1.0,0.9930556,2.6388889,0.0
,2019-01-31 13:30:55,3 min 36.561 sec,20.0,0.1064182,0.1077223,1.0,0.9984883,2.7112623,0.0,0.1094944,0.1104652,1.0,0.9930556,2.6388889,0.0
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2019-01-31 13:30:59,3 min 40.526 sec,335.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0277778,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.593 sec,340.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0277778,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.651 sec,345.0,0.0000000,0.0000000,1.0,0.0015117,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0208333,2.6388889,0.0
,2019-01-31 13:30:59,3 min 40.721 sec,350.0,0.0000000,0.0000000,1.0,0.0007559,2.7112623,0.0,0.0000001,0.0000000,1.0,0.0138889,2.6388889,0.0



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
features85_list_indices,2386.1770020,1.0,0.4569574
features85_list_values,1668.8812256,0.6993954,0.3195939
kl_words,580.0820923,0.2431010,0.1110868
ks_words,257.0541687,0.1077264,0.0492264
ngramscounts7_list_values,221.1662292,0.0926864,0.0423537
entropy_words,108.5201645,0.0454787,0.0207818


AttributeError: type object 'H2OGradientBoostingEstimator' has no attribute 'metalearner'

In [51]:
%matplotlib inline
metalearner.std_coef_plot()

NameError: name 'metalearner' is not defined

In [24]:
aml.leader.download_mojo(path = "./product_model_bin/ngrams7_features85_vectors_m25/v5/mojo", get_genmodel_jar = True)

'/home/siemanalyst/projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_bin/ngrams7_features85_vectors_m25/v5/mojo/GBM_grid_1_AutoML_20190131_132458_model_3.zip'

In [25]:
sc.stop()

In [26]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.tail(10))
print("prediction")
print(preds.tail(10))

gbm prediction progress: |████████████████████████████████████████████████| 100%
test


hash_message,fraud_label,kl_words,ks_words,entropy_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
7ff628bb5dad0bc3bfaace8dcaf726381c929567fd826cab34bcb1105bf7163d46bd0fedff87bb7f862c65062c6f66d3e180bc017dd8bcb86b7f0f339cca7e49,0,3138.88,0.821826,0.078551,0,85,0,262144,[ 1 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 40 41 42 43 44 45 46 55 57 60 63 64 73],[2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1.],[ 0 1 2 3 4 6 7 8 118 119 120 234 322 323 324 325 326 327 328 371 551 552 553 554 659 661 662 668 669 670 671 833 916 917 922 2099 4069 4072 4075 4078 4086 4094 4642 128603 154244 160075 189935 213892 223092 226684 237182 242787 243084 257059 259373],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
80f857718be06e2b40b853bca245aa2f837e3819078437c26f1d698066f3fb109b59a0c576124d19cf77ae4e84da945bf851a44ba51e1b7e5979f4b1743a269b,0,3329.22,1,0.0647077,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 51 52 53 54 56 59 61 62 64 65],[2. 2. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 32 33 34 45 46 47 48 49 50 51 52 53 54 66 67 68 69 70 303 304 536 537 538 539 540 742 759 760 761 825 1364 1367 1437 2438 3232 4499 22852 23023 23067 23271 23885 23986 24422 28670 28719 28815 29516 29863 30180 30888 31026 31316 31331 31824 32187 32268 32371 32423 168497 173647 186559],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
8304b24b1f8c03a69df330a2aa87776e29410e861b0fe6341c9ad39a3aeffcbb60fc0512edf219f2d222a2674955b6e6fbf9db39596c8e1643739f41c0e2be8f,0,6177.7,0.00655865,0.0662884,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 47 48 49 50 51 52 53 54 58 60 66 67 68 69 70 71 72 79 80],[4. 2. 3. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 9 19 20 21 22 23 24 25 28 29 30 31 35 55 58 59 79 80 81 82 83 84 85 86 87 88 89 93 94 95 96 97 98 99 100 101 102 103 106 107 108 149 197 198 199 216 217 409 410 411 107715 114476 131663 141917 143900 252020],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
83ab14f4f479c67c6604658cca2ae22c4ddaad756aaff0dc31ee0b1d2c17760d4d1a987fc6b8a704c0a17e02b841955152eabfb2973c5c844072290462fa56d2,0,5679.1,0.0642194,0.0708878,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 47 48 49 51 52 53 54 56 58 59 60 67 68 69 70 71 72 76 77 81 82 84],[5. 2. 3. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 9 10 11 12 19 20 21 22 23 24 25 26 27 28 29 35 36 37 43 44 104 109 110 111 112 113 114 115 116 136 137 138 139 140 141 142 143 150 153 154 155 156 157 158 159 160 161 162 163 164 165 166 173 174 175 301 302 308 46741 46756 50450 52504 52912 59145 59637 88987 102725 124273 145654 239935],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1


prediction


predict,p0,p1
0,1,2.51032e-11
0,1,2.43197e-11
0,1,3.43393e-13
0,1,9.69218e-12
0,1,5.20138e-11
0,1,6.69659e-12
0,1,1.26701e-11
0,1,1.46862e-10
0,1,1.46238e-12
0,1,2.00686e-10


In [27]:
sc.stop()

In [ ]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds_over_all_hf = aml.leader.predict(train)
path_out_file="20190122"
output_pred_file=h2o.export_file(frame=preds_over_all_hf, path=path_out_file, force=False)

gbm prediction progress: |